In [29]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd 
df = pd.read_csv("fina_Stage_unaugmented.csv")
from collections import Counter
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import numpy as np

C:\Users\khakh\AppData\Local\Temp\ipykernel_27540\650628951.py:4: DtypeWarning: Columns (29) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("fina_Stage_unaugmented.csv")


In [30]:
print(list(df.columns))

['Stage', 'Samples', 'oct_embedded', 'preservation_method', 'days_to_collection', 'initial_weight', 'pathology_report_uuid', 'synchronous_malignancy', 'laterality', 'treatments', 'tissue_or_organ_of_origin', 'age_at_diagnosis', 'primary_diagnosis', 'prior_malignancy', 'year_of_diagnosis', 'prior_treatment', 'method_of_diagnosis', 'ajcc_pathologic_t', 'morphology', 'ajcc_pathologic_n', 'ajcc_pathologic_m', 'icd_10_code', 'sites_of_involvement', 'race', 'ethnicity', 'vital_status', 'age_at_index', 'days_to_birth', 'demographic_id', 'age_is_obfuscated', 'paper_Included_in_previous_marker_papers', 'paper_vital_status', 'paper_days_to_birth', 'paper_days_to_last_followup', 'paper_age_at_initial_pathologic_diagnosis', 'paper_pathologic_stage', 'paper_BRCA_Subtype_PAM50', 'paper_CNV.Clusters', 'paper_Mutation.Clusters', 'paper_DNA.Methylation.Clusters', 'paper_mRNA.Clusters', 'paper_miRNA.Clusters', 'paper_PARADIGM.Clusters', 'TMEM220', 'SCN4B', 'CYYR1', 'LINC00478', 'MIR497HG', 'SPRY2', 'CAC

In [38]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.decomposition import PCA
from imblearn.over_sampling import SMOTE
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Input
from tensorflow.keras.callbacks import EarlyStopping
import joblib  # ‚Üê NEW

# Load and clean
df = pd.read_csv('fina_Stage_unaugmented.csv', low_memory=False)
df = df.drop_duplicates(subset='Samples')
df = df.dropna(subset=['Stage'])

# Drop unnecessary metadata
drop_cols = [
    'Samples', 'submitter_id', 'barcode', 'Unnamed: 0', 'sample_id',
    'sample', 'sample_submitter_id', 'patient', 'paper_patient',
    'diagnosis_id', 'bcr_patient_barcode', 'pathology_report_uuid',
    'treatments', 'releasable', 'released', 'paper_vital_status',
    'paper_Included_in_previous_marker_papers'
]
df = df.drop(columns=[col for col in drop_cols if col in df.columns], errors='ignore')

# Keep only numerical features + drop NaNs
y = df['Stage']
X = df.drop(columns=['Stage'])
X = X.select_dtypes(include=[np.number])
X.replace([np.inf, -np.inf], np.nan, inplace=True)
X.dropna(inplace=True)
y = y.loc[X.index]

# Encode labels
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)
num_classes = len(np.unique(y_encoded))

# Save label encoder
joblib.dump(label_encoder, "label_encoder.save")

# Scale + PCA
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
joblib.dump(scaler, "scaler.save")  # ‚Üê NEW

pca = PCA(n_components=0.95, random_state=42)
X_pca = pca.fit_transform(X_scaled)
joblib.dump(pca, "pca.save")  # ‚Üê NEW

# SMOTE
smote = SMOTE(random_state=42, k_neighbors=1)
X_resampled, y_resampled = smote.fit_resample(X_pca, y_encoded)

# One-hot target
y_onehot = tf.keras.utils.to_categorical(y_resampled, num_classes=num_classes)

# Split
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_onehot, test_size=0.2, random_state=42)

# ‚úÖ Vanilla NN Model
model = Sequential([
    Input(shape=(X_train.shape[1],)),
    Dense(128, activation='relu'),
    Dropout(0.3),
    Dense(64, activation='relu'),
    Dense(num_classes, activation='softmax')
])
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Training
history = model.fit(
    X_train, y_train,
    epochs=30,
    batch_size=32,
    validation_split=0.2,
    callbacks=[EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)],
    verbose=1
)

# Evaluate
loss, acc = model.evaluate(X_test, y_test)
print(f"\n‚úÖ Test Accuracy: {acc:.4f}")

# Save the full model
model.save("vanilla_nn_brca_model.keras")


Epoch 1/30
68/68 ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ 2s 11ms/step - accuracy: 0.3522 - loss: 2.1747 - val_accuracy: 0.7244 - val_loss: 0.8874
Epoch 2/30
68/68 ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ 0s 5ms/step - accuracy: 0.7669 - loss: 0.7621 - val_accuracy: 0.8045 - val_loss: 0.6048
Epoch 3/30
68/68 ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ 1s 6ms/step - accuracy: 0.8811 - loss: 0.4351 - val_accuracy: 0.8417 - val_loss: 0.4928
Epoch 4/30
68/68 ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ 0s 5ms/step - accuracy: 0.9247 - loss: 0.2749 - val_accuracy: 0.8510 - val_loss: 0.4474
Epoch 5/30
68/68 ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ 0s 5ms/step - accuracy: 0.9552 - loss: 0.1748 - val_accuracy: 0.8622 - val_loss: 0.4247
Epoch 6/30
68/68 ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ 0s 5ms/step - accuracy: 0.9642 - loss: 0.1336 - val_accuracy: 0.8603 - val_loss: 0.4530
Epo

In [17]:
import numpy as np
unique_test, counts_test = np.unique(np.argmax(y_test, axis=1), return_counts=True)
print(dict(zip(unique_test, counts_test)))


{np.int64(0): np.int64(14), np.int64(1): np.int64(16), np.int64(2): np.int64(61), np.int64(3): np.int64(42), np.int64(4): np.int64(24), np.int64(5): np.int64(4), np.int64(6): np.int64(10)}


In [32]:
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Model Accuracy Over Epochs')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

# Plot Loss
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Model Loss Over Epochs')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

plt.tight_layout()
plt.show()


In [11]:
from sklearn.metrics import confusion_matrix
import seaborn as sns
import numpy as np

cm = confusion_matrix(y_true_classes, y_pred_classes)
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=present_class_names,
            yticklabels=present_class_names)
plt.title('Confusion Matrix')
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.tight_layout()
plt.show()


NameError: name 'y_true_classes' is not defined

In [19]:
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import classification_report, confusion_matrix
import pandas as pd
import numpy as np

# Build classification report DataFrame
report = classification_report(
    y_true_classes, y_pred_classes,
    labels=present_labels,
    target_names=present_class_names,
    output_dict=True
)
df_report = pd.DataFrame(report).transpose()

# Create the full figure with 2 rows: top with 2 side-by-side, bottom full-width
fig = plt.figure(figsize=(16, 10))

# Top left: Accuracy
ax1 = fig.add_subplot(2, 2, 1)
ax1.plot(history.history['accuracy'], label='Train Accuracy')
ax1.plot(history.history['val_accuracy'], label='Validation Accuracy')
ax1.set_title('Model Accuracy Over Epochs')
ax1.set_xlabel('Epoch')
ax1.set_ylabel('Accuracy')
ax1.legend()
ax1.grid(True)

# Top right: Loss
ax2 = fig.add_subplot(2, 2, 2)
ax2.plot(history.history['loss'], label='Train Loss')
ax2.plot(history.history['val_loss'], label='Validation Loss')
ax2.set_title('Model Loss Over Epochs')
ax2.set_xlabel('Epoch')
ax2.set_ylabel('Loss')
ax2.legend()
ax2.grid(True)

# Bottom full row: F1-score
ax3 = fig.add_subplot(2, 1, 2)  # spans both columns
df_report.iloc[:-3]['f1-score'].plot(kind='bar', color='skyblue', ax=ax3)
ax3.set_title('F1-score by Cancer Stage')
ax3.set_ylabel('F1-score')
ax3.set_xlabel('Stage')
ax3.tick_params(axis='x', rotation=45)
ax3.grid(axis='y')

plt.tight_layout()
plt.show()


Now, we examine a combined visualization that illustrates the learning dynamics and classification strength of our vanilla neural network. This figure is organized into two main sections: the top row captures the model‚Äôs training history (accuracy and loss), and the bottom row displays the F1-scores across each breast cancer stage class a crucial performance metric in this multi-class, imbalanced dataset. In the top-left subplot, the accuracy curves show how well the model fits the data over time. We observe that training accuracy improves steadily, rising from around 45% in the first epoch to over 98% by epoch 11. Meanwhile, validation accuracy increases more moderately, plateauing just below 90%. This trend indicates that the model is learning generalizable patterns in the data without immediately overfitting. However, the growing gap between training and validation accuracy after epoch 5 suggests that the model is starting to over-specialize on the training set ‚Äî a known risk in deep learning, especially with high-capacity models trained on finite data.

The top-right plot shows a similar pattern in terms of loss. Both training and validation losses decrease sharply during the early epochs, reflecting rapid convergence. After this point, training loss continues to fall toward zero, but validation loss levels off and begins to slightly rise. This inflection point confirms the beginning of overfitting. Thankfully, regularization techniques like dropout, early stopping, and learning rate reduction were integrated into the model‚Äôs training loop to mitigate this and halt the training process before the model fully memorizes the training data. The F1-score bar chart in the second row presents a breakdown of classification performance across all 13 cancer stages. The F1-score combines precision and recall, making it ideal for evaluating model performance on imbalanced classes where accuracy alone can be misleading. In this plot, we see that the model performs exceptionally well on several stages, achieving F1-scores close to 1.0. However, performance varies across the board.

['Stage', 'Samples', 'oct_embedded', 'preservation_method', 'days_to_collection', 'initial_weight', 'pathology_report_uuid', 'synchronous_malignancy', 'laterality', 'treatments', 'tissue_or_organ_of_origin', 'age_at_diagnosis', 'primary_diagnosis', 'prior_malignancy', 'year_of_diagnosis', 'prior_treatment', 'method_of_diagnosis', 'ajcc_pathologic_t', 'morphology', 'ajcc_pathologic_n', 'ajcc_pathologic_m', 'icd_10_code', 'sites_of_involvement', 'race', 'ethnicity', 'vital_status', 'age_at_index', 'days_to_birth', 'demographic_id', 'age_is_obfuscated', 'paper_Included_in_previous_marker_papers', 'paper_vital_status', 'paper_days_to_birth', 'paper_days_to_last_followup', 'paper_age_at_initial_pathologic_diagnosis', 'paper_pathologic_stage', 'paper_BRCA_Subtype_PAM50', 'paper_CNV.Clusters', 'paper_Mutation.Clusters', 'paper_DNA.Methylation.Clusters', 'paper_mRNA.Clusters', 'paper_miRNA.Clusters', 'paper_PARADIGM.Clusters', 'TMEM220', 'SCN4B', 'CYYR1', 'LINC00478', 'MIR497HG', 'SPRY2', 'CAC

TMEM220      SCN4B      CYYR1  LINC00478   MIR497HG      SPRY2  \
0     2.087876   4.136425  24.841505   0.645430   4.612107   7.174804   
2     5.439423  10.021643  19.771319   3.468410   9.747095  30.387948   
4     2.591338   3.812829  12.153050   2.063287   3.434179   7.197078   
6     4.061954   8.903831  16.749300   3.761841  10.394742  35.908754   
8     1.346772   1.650577   9.087307   2.695105   1.767846   3.791904   
...        ...        ...        ...        ...        ...        ...   
2189  1.623698   4.078569  59.688666   0.730480   8.139201  32.982142   
2191  1.664184   3.828402   9.470195   2.233674   2.792487   8.347097   
2193  1.753617   8.067380   5.812520   0.615198   8.882094  11.505114   
2195  0.893142   2.472041   7.315998   3.802674   3.273487   5.081274   
2197  5.530075   9.883019  49.331907   5.255076   8.075720  33.868546   

         CACHD1   SH3BGRL2      SPC25       NEK2  ...       TUBA1B  \
0      5.196903   4.802379   8.975385  19.337598  ...   888.126605   
2     17.346754  13.093555  11.146207  10.208067  ...  1066.941985   
4     12.535631  16.207475  11.375695  19.309219  ...   749.730287   
6     10.419887  20.423754  12.225548  12.651472  ...   887.962496   
8      2.386469   2.668765  11.749391  10.718309  ...  1655.063781   
...         ...        ...        ...        ...  ...          ...   
2189   4.546981   7.909083  20.236808   7.830319  ...  1344.779695   
2191  10.502283   5.015472  23.497009  18.016743  ...  1450.712842   
2193   1.841167   2.639245  33.806698  32.191787  ...  1798.347566   
2195   2.525718   4.219095   6.959707   5.954067  ...   971.804840   
2197  21.487918  18.611863  20.229038   7.521767  ...  1036.710997   

           RBPMS  LOC100129722    CASP14    PSAPL1      DSPP    ARMC12  \
0      34.676645      0.355111  0.276181  0.008067  0.000000  0.226064   
2      49.057113      0.482939  0.467562  0.027715  0.029949  1.109557   
4      47.416268      0.739856  2.041451  0.026891  0.000000  0.538278   
6     168.054265      1.295903  0.309799  0.065722  0.000000  0.438524   
8      36.784990      0.110027  0.744100  0.000000  0.000000  0.120074   
...          ...           ...       ...       ...       ...       ...   
2189   44.366293      0.188393  0.016988  0.011412  0.000000  1.005137   
2191   57.536202      0.073782  0.053225  0.000000  0.000000  0.679947   
2193   38.715767      2.145931  1.127198  0.007572  0.024548  0.848841   
2195   45.950714      0.000000  1.119491  0.000000  0.000000  0.370566   
2197   63.447540      0.027316  0.044336  0.029785  0.032185  0.516716   

        IL31RA       BCL9     CWH43  
0     0.020568  10.587121  0.030569  
2     0.035333  18.793346  0.175043  
4     0.034282  34.376663  3.108023  
6     0.097751  18.762315  0.290562  
8     0.076473  32.117110  1.269175  
...        ...        ...       ...  
2189  0.072745  31.963884  0.129739  
2191  0.034188  28.056393  0.016937  
2193  0.028961  27.324705  0.688695  
2195  0.000000  16.116987  0.065768  
2197  0.025314  32.101772  0.094058  

[1013 rows x 2000 columns]

generate sample

In [33]:
import pandas as pd

# Load full dataset
df = pd.read_csv("fina_Stage_unaugmented.csv", low_memory=False)

# Drop NaN and duplicates (same cleaning as training)
df = df.drop_duplicates(subset='Samples')
df = df.dropna(subset=['Stage'])

# Drop columns not used in the final model
drop_cols = [
    'Samples', 'submitter_id', 'barcode', 'Unnamed: 0', 'sample_id',
    'sample', 'sample_submitter_id', 'patient', 'paper_patient',
    'diagnosis_id', 'bcr_patient_barcode', 'pathology_report_uuid',
    'treatments', 'Stage'  # don't include target when predicting
]
df = df.drop(columns=[col for col in drop_cols if col in df.columns], errors='ignore')

# Keep only numeric columns
df = df.select_dtypes(include=[float, int])
df.replace([np.inf, -np.inf], pd.NA, inplace=True)
df.dropna(inplace=True)

# Pick 1 random sample and save
sample = df.sample(n=1, random_state=42)
sample.to_csv("random_sample2.csv", index=False)
print("‚úÖ Saved 1 random sample to random_sample.csv")


‚úÖ Saved 1 random sample to random_sample.csv


In [34]:
import pandas as pd
import numpy as np
import joblib
from tensorflow.keras.models import load_model

# Load sample
sample = pd.read_csv("random_sample.csv")

# Load saved preprocessor and model
scaler = joblib.load("scaler.save")
pca = joblib.load("pca.save")
label_encoder = joblib.load("label_encoder.save")
model = load_model("vanilla_nn_brca_model.keras")

# Preprocess sample
sample_scaled = scaler.transform(sample)  # Use transform, not fit_transform
sample_pca = pca.transform(sample_scaled)

# Predict
pred_probs = model.predict(sample_pca)
pred_index = np.argmax(pred_probs)
pred_class = label_encoder.inverse_transform([pred_index])[0]

print(f"üîç Predicted Stage: {pred_class}")
print("\nüìä Stage Probabilities:")
for stage, prob in zip(label_encoder.classes_, pred_probs[0]):
    print(f"{stage}: {prob*100:.2f}%")

1/1 ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ 0s 116ms/step
üîç Predicted Stage: Stage IIA

üìä Stage Probabilities:
Stage I: 0.01%
Stage IA: 0.02%
Stage IB: 0.07%
Stage II: 0.00%
Stage IIA: 98.85%
Stage IIB: 0.77%
Stage IIIA: 0.11%
Stage IIIB: 0.00%
Stage IIIC: 0.15%
Stage IV: 0.01%
Stage X: 0.00%


In [26]:
print(list(pd.read_csv("test_sample.csv").columns))

['Samples', 'oct_embedded', 'preservation_method', 'days_to_collection', 'initial_weight', 'pathology_report_uuid', 'synchronous_malignancy', 'laterality', 'treatments', 'tissue_or_organ_of_origin', 'age_at_diagnosis', 'primary_diagnosis', 'prior_malignancy', 'year_of_diagnosis', 'prior_treatment', 'method_of_diagnosis', 'ajcc_pathologic_t', 'morphology', 'ajcc_pathologic_n', 'ajcc_pathologic_m', 'icd_10_code', 'sites_of_involvement', 'race', 'ethnicity', 'vital_status', 'age_at_index', 'days_to_birth', 'demographic_id', 'age_is_obfuscated', 'paper_Included_in_previous_marker_papers', 'paper_vital_status', 'paper_days_to_birth', 'paper_days_to_last_followup', 'paper_age_at_initial_pathologic_diagnosis', 'paper_pathologic_stage', 'paper_BRCA_Subtype_PAM50', 'paper_CNV.Clusters', 'paper_Mutation.Clusters', 'paper_DNA.Methylation.Clusters', 'paper_mRNA.Clusters', 'paper_miRNA.Clusters', 'paper_PARADIGM.Clusters', 'TMEM220', 'SCN4B', 'CYYR1', 'LINC00478', 'MIR497HG', 'SPRY2', 'CACHD1', 'SH